<a href="https://colab.research.google.com/github/jaragos0s/deep_learning/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

핸즈온 머신러닝 
#Chapter16. RNN과 어텐션을 사용한 자연어 처리

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Char-RNN을 사용해 셰익스피어 문체 생성하기

In [2]:
import sys
import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


In [4]:
'''작품 다운로드'''
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [6]:
print(shakespeare_text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [7]:
'''shakespeare.txt 파일에 어떤 문자가 있는지 sorting 해서 출력'''
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [8]:
''' 모든 글자를 정수로 인코딩 '''
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True) # char_level = True -> 단어 수준 인코딩(기본적으로 텍스트를 소문자로 바꿈)
tokenizer.fit_on_texts(shakespeare_text)

In [9]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [10]:
max_id = len(tokenizer.word_index) # 고유 글자 개수
dataset_size = tokenizer.document_count # 전체 글자 개수

In [12]:
''' 전체 텍스트를 인코딩해 각 글자를 ID로 나타냄 '''
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

### 순차 데이터셋 나누기

훈련 세트, 검증 세트, 테스트 세트가 중복되지 않도록 만들어야 한다. 

시계열 데이터는 보통 시간에 따라 나눈다. 


In [13]:
''' 텍스트의 처음 90%를 훈련 세트로 사용 '''
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### 순차 데이터를 윈도 여러 개로 자르기